In [ ]:
# Import packages
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import altair as alt

In [ ]:
# Run authentication
ee.Authenticate()
ee.Initialize(project='') # PUT YOUR API KEY (Project ID) HERE!

# Change detection

**What's the goal today?**

Today we are going to work with a few series of optical data at different points in time. Using this data, we will attempt to identify different changes in the landscape, and then try to understand how this can be useful. We will also discuss the limitations, restrictions and problems that we may or do encounter, and how we could resolve these.

# Scenario 1 - detecting the evolution of Supraglacial Lakes in Antarctica

Supraglacial lakes are a phenomenon that are thought to indicate the decay of ice shelves on continental scale ice caps. They typically are only found in Antarctica, Greenland and a few in smaller ice masses. They are relatively easy to detect using spectral indices, and we can analyse how they evolve over time to determine a general overview of the "health" of the ice shelf. Typically, supraglacial lakes are problematic for ice masses in several ways, by affecting their albedo, fracturing the ice and enhanced melting - but today we are simply interested in how they change over time.

We can look at this from two angles:


1.   How supraglacial lakes change throughout a single melt season
2.   How supraglacial lakes change between melt seasons

In this example, we will approach a comparison of two different melt seasons on the Antarctic Peninsula.



In [ ]:
## Overview
# Produce and overview map of the Antarctic Peninsula, focused closely on George VI ice shelf, use global imagery
Overdata = ee.ImageCollection("USGS/LIMA/SR")
AntarcticaIMG = Overdata.select(['B3', 'B2', 'B1']).mosaic() # Mosaic to get a single image
Overview = geemap.Map()

# Define visualization parameters for better visibility
vis_params = {
    'min': 0.0,
    'max': 10000, # Adjust max value to improve brightness
    'gamma': 1.0 # Adjust gamma for contrast enhancement
}

Overview.addLayer(AntarcticaIMG, vis_params, 'Antarctica')

# Define the bounding box for George VI ice shelf (example coordinates, adjust as needed)
george_vi_bbox = ee.Geometry.Rectangle([-70.5, -73.5, -66.5, -70.5])

# Add the bounding box as a layer with translucent fill and red outline
Overview.addLayer(george_vi_bbox, {'color': 'red'}, 'George VI Ice Shelf Bounding Box')

Overview

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
## Import satellite scenes

# Import Sentinel-2 collection
sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR')

# Define the bounding box for George VI ice shelf (same as before)
george_vi_bbox = ee.Geometry.Rectangle([-70.5, -73.5, -66.5, -70.5])

# Filter Sentinel-2 imagery for January 2020 and the bounding box
sentinel2_jan_2020 = sentinel2.filterDate('2020-01-01', '2020-01-31').filterBounds(george_vi_bbox)

# Create a mosaic of the filtered imagery
jan_2020_mosaic = sentinel2_jan_2020.mosaic()

# Define visualization parameters for true color (B4, B3, B2)
vis_params_true_color = {
    'min': 0,
    'max': 50000,
    'gamma': 1.5,
    'bands': ['B4', 'B3', 'B2']
}

# Create a new map and add the mosaic layer
SentinelMap = geemap.Map()
SentinelMap.addLayer(jan_2020_mosaic, vis_params_true_color, 'Sentinel-2 January 2020')

# Display the new map
SentinelMap

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# What's the problem here?

Cloud cover!

Even though there's available imagery, clouds cause problems for what we are seeing - a lot of imagery becomes restricted based on clouds obstructing our view of the site.

This is a consistent problem with George VI ice shelf, so whilst it is good to think of alternatives for avoiding clouds, we are going to pivot across the Antarctic Peninsula to the Wilkins ice shelf.

In [ ]:
## Import and display Sentinel-2 mosaics for January 2022 and 2025 over Wilkins Ice Shelf and save as variables

# Import Sentinel-2 collection
sentinel2_collection = ee.ImageCollection('COPERNICUS/S2_SR')

# Define the bounding box for Wilkins Ice Shelf (example coordinates, adjust as needed)
wilkins_bbox = ee.Geometry.Rectangle([-73.5, -70.5, -71.5, -69.5])


# Define the specific years to process
years = [2022, 2025]

# Define visualization parameters for true color (B4, B3, B2)
vis_params_true_color = {
    'min': 0,
    'max': 50000, # Set max visual parameter to 50000 for Sentinel
    'gamma': 1.5,
    'bands': ['B4', 'B3', 'B2']
}

# Create a new map to display all layers
MultiSeasonMap = geemap.Map()

# Process each January and save the mosaic to a variable
for year in years:
    start_date = f'{year}-01-01'
    end_date = f'{year}-01-31'

    # Filter Sentinel-2 imagery for January of the year, bounding box, and less than 20% cloud cover
    filtered_collection = sentinel2_collection.filterDate(start_date, end_date).filterBounds(wilkins_bbox).filterMetadata('CLOUD_COVERAGE_ASSESSMENT', 'less_than', 20)

    # Sort the collection by cloud cover to minimize overlaps
    sorted_collection = filtered_collection.sort('CLOUD_COVERAGE_ASSESSMENT')

    # Create a mosaic
    mosaic = sorted_collection.mosaic()

    # Assign the mosaic to a year-specific variable
    if year == 2022:
        Wilkins_2022 = mosaic
        layer_name = 'Sentinel-2 January 2022 Wilkins IS < 20% Cloud Cover'
    elif year == 2025:
        Wilkins_2025 = mosaic
        layer_name = 'Sentinel-2 January 2025 Wilkins IS < 20% Cloud Cover'
    else:
        layer_name = f'Sentinel-2 January {year} Wilkins IS < 20% Cloud Cover' # Fallback for other years if added


    # Add the mosaic as a layer to the map
    MultiSeasonMap.addLayer(mosaic, vis_params_true_color, layer_name)

# Display the map
MultiSeasonMap

# Note: The mosaics for 2022 and 2025 are now available in the variables 'Wilkins_2022' and 'Wilkins_2025'.
# You can toggle the visibility of each layer using the layer control panel on the map.
# The layers are added in the order they appear in the loop, with the last one on top.

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# Wilkins ice shelf

Have a look at this ice shelf, investigate anything interesting you find on the surface. You should be able to find some blue patches.

These are supraglacial lakes. You should also have two sets of data for this site, one from 2021-2022 and one from 2024-2025's melt seasons in January. Compared to George VI, there are fewer lakes here. But, something we can analyse is how they change between these two snapshots. Have a look, see if you can notice any immediate differences.

Whilst it is relatively easy to see some minor differences with manual study, we can also try something more automated. Here, we can try some spectral indices to compare the lake areas in each period.

In [ ]:
## Automating our ability to find the differences using Red-Blue NDWI
# Data required here:
# Variables - Wilkins_2022 (Image) and Wilkins_2025 (Image)

# Calculate Red-Blue NDWI for Wilkins_2022: (B2 - B4) / (B2 + B4)
# Note: Use .normalizedDifference(['B2', 'B4']) which calculates (B2 - B4) / (B2 + B4)
ndwi_wilkins_2022 = Wilkins_2022.normalizedDifference(['B2', 'B4']).rename('RB_NDWI_2022')

# Calculate Red-Blue NDWI for Wilkins_2025: (B2 - B4) / (B2 + B4)
ndwi_wilkins_2025 = Wilkins_2025.normalizedDifference(['B2', 'B4']).rename('RB_NDWI_2025')

# Define visualization parameters for the NDWI (typically ranges from 0 to 0.1 on ice here for good visualisation purposes)
# Water usually has positive NDWI values, adding gamma as requested and removing palette.
ndwi_vis_params = {
    'min': 0, # Adjust min/max for better visualization of water features
    'max': 0.1,
    'gamma': 1.0 # Added gamma parameter
}

# Create a new map to display the NDWI layers
NDWIMap = geemap.Map()

# Add the NDWI layers to the map
NDWIMap.addLayer(ndwi_wilkins_2022, ndwi_vis_params, 'Red-Blue NDWI 2022')
NDWIMap.addLayer(ndwi_wilkins_2025, ndwi_vis_params, 'Red-Blue NDWI 2025')


# Display the map
NDWIMap

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# What next?

Now, we have some indexed values - areas of lakes show up pretty well. However, so does a lot of other stuff. Have a think of some ways which we could use to remove this from our output - this is something to discuss and consider for your projects!

If we want to see the change between each image, we should continue with the script though, as this just makes the changes easier to visualise but still involves flipping through images.

In [ ]:
## Calculate and display the difference between 2025 and 2022 Red-Blue NDWI

# Data required here:
# Variables - ndwi_wilkins_2022 (Image) and ndwi_wilkins_2025 (Image)

# Calculate the difference image: 2025 NDWI - 2022 NDWI
ndwi_difference = ndwi_wilkins_2025.subtract(ndwi_wilkins_2022).rename('RB_NDWI_Difference')

# Define visualization parameters for the difference image
# This will highlight positive changes (increase in NDWI) and negative changes (decrease in NDWI)
# The greater the values of "min" and "max" the greater the clearly visible differences will be.
# If we only have -0.1 to 0.1 for example, we will see every tiny difference, and the base colour
# ramp is not very good for that purpose...
difference_vis_params = {
    'min': -0.5,  # Adjust min/max to focus on significant changes
    'max': 0.5,
    'palette': ['FF0000', 'FFFFFF', '0000FF'] # Red for decrease, White for no change, Blue for increase
}

# Create a new map and add the difference layer
DifferenceMap = geemap.Map()

# Add the NDWI difference layer to the map
DifferenceMap.addLayer(ndwi_difference, difference_vis_params, 'Red-Blue NDWI Difference (2025 - 2022)')


# Display the map
DifferenceMap

# Note: Red areas indicate a decrease in NDWI (e.g., water shrinking or disappearing),
# Blue areas indicate an increase in NDWI (e.g., new or expanding water bodies),
# and white/grey areas indicate little to no change.

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# Try something else!

Now that you hhave seen this example, it's good to get thinking about other examples and case studies which you could use this analysis on. It's particularly useful for your projects as these will be kicked off soon.

Some examples you could try:


*   Tsunami impact in Alaska (2025 Tracy Arm Landslide)
*   Tracking lava flows on Mt. Etna in Italy (snow and vegetation changes)
*   Flooding impact in Norway during Storm Hans August 2023 (Water detection)
*   Seasonal or annual snow line changes (Snow indices)

